In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB, Bnd_Box
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire, TopoDS_Vertex, TopoDS_Shape, TopoDS_Face
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Ax1, gp_Dir, gp_Pln, gp_Vec, gp_Mat, gp_Quaternion
from OCC.Display.SimpleGui import init_display
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from IPython.display import display, HTML
from math import acos, degrees
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Geom import Geom_CylindricalSurface

import pandas as pd
import numpy as np
import json
import math, os

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# paths
json_path = "../data/Shape_classifier_info.json"
nc_path = "../data/output/nc1/"
drawing_path = "../data/output/drawings/"
media_path = "../data/media/"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.AIS import AIS_Shape

def visualize(shape, display, color="CYAN", alpha=None, clear=False):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        display: OCC viewer handle.
        color: Optional string color name (default "CYAN").
        alpha: Optional float (0.0 - 1.0) for transparency (0 = fully transparent, 1 = opaque).
        clear: If True, erase all previous shapes before displaying.
    """

    def get_occ_color(color_name):
        colors = {
            "RED": (1.0, 0.0, 0.0),
            "GREEN": (0.0, 1.0, 0.0),
            "BLUE": (0.0, 0.0, 1.0),
            "PSS_DARK_BLUE": (0.023, 0.105, 0.215),
            "PSS_LIGHT_BLUE": (0.592, 0.792, 0.921),
            "YELLOW": (1.0, 1.0, 0.0),
            "ORANGE": (1.0, 0.5, 0.0),
            "CYAN": (0.0, 1.0, 1.0),
            "MAGENTA": (1.0, 0.0, 1.0),
            "WHITE": (1.0, 1.0, 1.0),
            "GRAY": (0.5, 0.5, 0.5),
            "BLACK": (0.0, 0.0, 0.0),
        }
        r, g, b = colors.get(color_name.upper(), (1.0, 1.0, 1.0))
        return Quantity_Color(r, g, b, Quantity_TOC_RGB)

    if clear:
        display.Context.EraseAll()

    # Convert color name to OCC color
    qcolor = get_occ_color(color)

    # Wrap in AIS_Shape to support transparency
    ais_shape = AIS_Shape(shape)
    ais_shape.SetColor(qcolor)

    if alpha is not None:
        ais_shape.SetTransparency(1.0 - alpha)  # OCC expects "opacity", not alpha
        ais_shape.SetDisplayMode(1)  # Shaded

    display.Context.Display(ais_shape, True)
    display.FitAll()


In [4]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb['center']
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb['half_lengths'][0], obb['half_lengths'][1], obb['half_lengths'][2]
    dx, dy, dz = obb['dir_x'], obb['dir_y'], obb['dir_z']

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [5]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def robust_align_solid_from_geometry(solid, tol=1e-3):
    """
    Aligns a solid such that:
    - X axis is the longest edge on the largest planar face
    - Y axis is the normal of that face
    - Z axis is X × Y (right-handed)
    Returns:
    - aligned solid
    - transformation used
    - local coordinate system (gp_Ax3)
    - largest face used
    - dir_x, dir_y, dir_z
    """
    # Step 1: Find the largest planar face
    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    largest_face = None
    max_area = 0.0

    while explorer.More():
        face = explorer.Current()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        area = props.Mass()
        if area > max_area:
            max_area = area
            largest_face = face
        explorer.Next()

    if not largest_face:
        raise RuntimeError("No planar face found in solid.")

    # Step 2: Get longest edge on that face → X axis
    edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
    longest_vec = None
    max_length = 0.0

    while edge_exp.More():
        edge = edge_exp.Current()
        curve = BRepAdaptor_Curve(edge)
        p1 = curve.Value(curve.FirstParameter())
        p2 = curve.Value(curve.LastParameter())
        vec = gp_Vec(p1, p2)
        length = vec.Magnitude()
        if length > max_length:
            max_length = length
            longest_vec = vec
        edge_exp.Next()

    if not longest_vec or longest_vec.Magnitude() < tol:
        raise RuntimeError("Failed to find valid longest edge.")

    dir_x = gp_Dir(longest_vec)

    # Step 3: Get face normal → Y axis
    surf_adapt = BRepAdaptor_Surface(largest_face)
    if surf_adapt.GetType() != 0:  # not a plane
        raise RuntimeError("Largest face is not planar.")
    dir_y = surf_adapt.Plane().Axis().Direction()

    # Step 4: Compute Z = X × Y, and re-orthogonalize Y to X and Z
    dir_z = dir_x.Crossed(dir_y)
    dir_y = dir_z.Crossed(dir_x)  # ensure orthogonality

    # Step 5: Create transformation from this local frame to global (X,Y,Z)
    origin = gp_Pnt(0, 0, 0)
    from_cs = gp_Ax3(origin, dir_x, dir_y)
    to_cs = gp_Ax3(origin, gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))  # global X/Y/Z

    trsf = gp_Trsf()
    trsf.SetDisplacement(from_cs, to_cs)

    aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()
    
    return aligned, trsf, to_cs, largest_face, dir_x, dir_y, dir_z


In [6]:
from OCC.Core.gp import gp_Ax3, gp_Trsf, gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_to_dstv_frame(solid, origin_local, dir_x, dir_y, dir_z):
    """
    Transforms a solid so that:
    - dir_x aligns to global X (beam length)
    - dir_y aligns to global Y (beam height)
    - dir_z aligns to global Z (beam width)
    - origin_local maps to global (0, 0, 0)
    """
    # Create local coordinate system from solid geometry
    local_ax3 = gp_Ax3(origin_local, gp_Dir(*dir_x), gp_Dir(*dir_y))

    # Create global DSTV coordinate system
    global_ax3 = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))

    # Create transformation
    trsf = gp_Trsf()
    trsf.SetDisplacement(local_ax3, global_ax3)

    # Apply to shape
    transformer = BRepBuilderAPI_Transform(solid, trsf, True)
    return transformer.Shape(), trsf


In [7]:
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir

def compute_dstv_origin(center: gp_Pnt, extents: list, dir_x: gp_Dir, dir_y: gp_Dir, dir_z: gp_Dir):
    """
    Computes the DSTV origin (rear-bottom-left) for a shape aligned via OBB.

    Parameters:
    - center: gp_Pnt — the center of the OBB (post-alignment)
    - extents: [length, height, width] — full extents along dir_x, dir_y, dir_z
    - dir_x: gp_Dir — length/feed direction
    - dir_y: gp_Dir — height/web direction
    - dir_z: gp_Dir — width/flange direction

    Returns:
    - gp_Pnt — rear-bottom-left corner in the aligned local frame
    """

    # Compute offset from center to rear-bottom-left corner
    dx = -extents[0] / 2  # back along length
    dy = -extents[1] / 2  # down along height
    dz = -extents[2] / 2  # left along width

    # Offset vector from center to origin
    offset_vec = gp_Vec(dir_x).Scaled(dx) + gp_Vec(dir_y).Scaled(dy) + gp_Vec(dir_z).Scaled(dz)

    # Translate center to origin
    origin_local = center.Translated(offset_vec)
    return origin_local


In [8]:
# import json

def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to detect if a 90° rotation is needed.
    """
    with open(json_path) as f:
        lib = json.load(f)

    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        "Designation": name,
                        "Category": cat,
                        "Profile_type": info["code_profile"],
                        "Match_score": score,
                        "Requires_rotation": False,  # to be set later
                    
                        "JSON": {
                            "height": info["height"],
                            "width": info["width"],
                            "csa": info["csa"],
                            "length": 0,
                            "Mass": info.get("mass", 0.0)*cs["length"]/1000,
                            "web_thickness": info.get("web_thickness", 0.0),
                            "flange_thickness": info.get("flange_thickness", 0.0),
                            "root_radius": info.get("root_radius", 0.0),
                            "toe_radius":  info.get("toe_radius", 0.0)
                        },
                        "STEP": {
                            "height": height,
                            "width": width,
                            "area": cs["area"],
                            "length": info.get("length", cs["length"]),
                            "mass": info.get("mass", 0.0)*cs["length"]/1000,
                            "web_thickness": info.get("web_thickness", 0.0),
                            "flange_thickness": info.get("flange_thickness", 0.0),
                            "root_radius": info.get("root_radius", 0.0),
                            "toe_radius":  info.get("toe_radius", 0.0)
                        }
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    header_dict = {


    }
   
    if original and (not swapped or score1 <= score2):
        original["Requires_rotation"] = False
        return original
    elif swapped:
        swapped["Requires_rotation"] = True
        return swapped
    else:
        return None


In [9]:
def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    step_vals = result["STEP"]
    json_vals = result["JSON"]
    
    data = {
        "Field": [
            "Designation", "Category", "Profile Type", "Match Score", "Rotation Required",
            "Height (mm)", "Width (mm)", "CSA (mm²)", "Length (mm)", "Mass (Kg)",
            "Web Thickness (mm)", "Flange Thickness (mm)", "Root Radius (mm)", "Toe Radius (mm)"
        ],
        "STEP File": [
            result["Designation"],
            result["Category"],
            result["Profile_type"],
            f"{result['Match_score']:.2f}",
            result["Requires_rotation"],
            f"{step_vals['height']:.2f}",
            f"{step_vals['width']:.2f}",
            f"{step_vals['area']:.2f}",
            f"{step_vals['length']:.2f}",
            f"{step_vals['mass']:.2f}",
            "","","",""
        ],
        "JSON Spec": [
            "", "", "", "", "",
            f"{json_vals['height']:.2f}",
            f"{json_vals['width']:.2f}",
            f"{json_vals['csa']:.2f}",
            f"{json_vals['length']:.2f}",
            "",
            f"{json_vals['web_thickness']:.2f}",
            f"{json_vals['flange_thickness']:.2f}",
            f"{json_vals['root_radius']:.2f}",
            f"{json_vals['toe_radius']:.2f}"
        ]
    }

    df = pd.DataFrame(data)
    display(df.style.set_properties(**{
        'text-align': 'left'
    }).set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }]))



In [10]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

def compute_section_area(shape):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(shape, props)
    return props.Mass()

def compute_axis_aligned_bounding_box(shape):
    """
    Computes a world-aligned bounding box (AABB) and includes true section area.
    """
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()

    # Half-sizes
    hx = (xmax - xmin) / 2
    hy = (ymax - ymin) / 2
    hz = (zmax - zmin) / 2

    center = gp_Pnt((xmin + xmax)/2, (ymin + ymax)/2, (zmin + zmax)/2)
    
    # Define frame as default world axes
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)
    ax3 = gp_Ax3(center, dir_z, dir_x)

    # Compute actual section area
    csa = compute_section_area(shape)

    return {
        "frame": ax3,
        "center": center,
        "dir_x": dir_x,
        "dir_y": dir_y,
        "dir_z": dir_z,
        "half_x": hx,
        "half_y": hy,
        "half_z": hz,
        "extents": [2*hx, 2*hy, 2*hz],
        "section_area": csa
    }


In [11]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt

def make_origin_marker(point=gp_Pnt(0, 0, 0), radius=5.0):
    """
    Creates a sphere centered at the global origin for visualization.
    """
    return BRepPrimAPI_MakeSphere(point, radius).Shape()


In [12]:
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def make_arrow(start: gp_Pnt, direction: gp_Dir, length=100.0):
    """
    Creates a line-shaped arrow from a start point in the given direction.
    """
    vec = gp_Vec(direction) * length
    end = gp_Pnt(start.XYZ())
    end.Translate(vec)
    return BRepBuilderAPI_MakeEdge(start, end).Shape()


In [13]:
from OCC.Core.gp import gp_Ax3, gp_Dir, gp_Pnt, gp_Trsf, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_obb_to_dstv_frame(shape, origin_local, dir_x, dir_y, dir_z):
    """
    Align a solid into the DSTV frame:
    1. Translates the origin to (0, 0, 0)
    2. Rotates the axes to X = length, Y = height, Z = width (DSTV frame)

    Parameters:
    - shape: the OCC shape to transform
    - origin_local: gp_Pnt, the local rear-bottom-left corner
    - dir_x, dir_y, dir_z: gp_Dir, aligned local axes (length, height, width)

    Returns:
    - Transformed shape aligned with DSTV frame
    - Final rotation transform
    """

    # STEP 1: Translate shape so origin_local becomes (0, 0, 0)
    translate_vec = gp_Vec(origin_local, gp_Pnt(0, 0, 0))
    trsf_translate = gp_Trsf()
    trsf_translate.SetTranslation(translate_vec)

    shape_translated = BRepBuilderAPI_Transform(shape, trsf_translate, True).Shape()

    # STEP 2: Rotate to align local frame with DSTV global frame
    # Local frame with current axes
    local_frame = gp_Ax3(gp_Pnt(0, 0, 0), dir_z, dir_x)  # Z = up, X = length
    # Target DSTV frame
    dstv_frame = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))  # Z up, X forward

    trsf_rotate = gp_Trsf()
    trsf_rotate.SetDisplacement(local_frame, dstv_frame)

    shape_rotated = BRepBuilderAPI_Transform(shape_translated, trsf_rotate, True).Shape()

    return shape_rotated, trsf_rotate


In [14]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir

def compute_obb_geometry(aligned_shape):
    """
    Compute extents and axes of a shape aligned in its own OBB frame.
    Returns length, height, width and the direction vectors.
    """
    # Step 1: Get the bounding box in the aligned frame
    aabb = Bnd_Box()
    brepbndlib.Add(aligned_shape, aabb)
    xmin, ymin, zmin, xmax, ymax, zmax = aabb.Get()

    # Step 2: Compute dimensions
    length = xmax - xmin
    height = ymax - ymin
    width = zmax - zmin

    # Step 3: Use world axes since shape is aligned
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)

    # Step 4: Compute center
    center = gp_Pnt(
        (xmin + xmax) / 2,
        (ymin + ymax) / 2,
        (zmin + zmax) / 2
    )

    return {
        "aligned_extents": [length, height, width],
        "aligned_dir_x": dir_x,
        "aligned_dir_y": dir_y,
        "aligned_dir_z": dir_z,
        "aligned_center": center
    }


In [15]:
import numpy as np

def ensure_right_handed(dir_x, dir_y, dir_z):
    x = np.array([dir_x.X(), dir_x.Y(), dir_x.Z()])
    y = np.array([dir_y.X(), dir_y.Y(), dir_y.Z()])
    z = np.array([dir_z.X(), dir_z.Y(), dir_z.Z()])

    if np.dot(np.cross(x, y), z) < 0:
        print("⚠️ Detected left-handed system. Reversing Z to enforce right-handed convention.")
        dir_z = gp_Dir(-dir_z.X(), -dir_z.Y(), -dir_z.Z())

    return dir_x, dir_y, dir_z


In [16]:
def compute_section_area(solid):
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    volume = props.Mass()

    bbox = Bnd_Box()
    brepbndlib.Add(solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    length = xmax - xmin  # assuming length is X after alignment

    area = volume / length if length > 0 else 0
    return area


In [17]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Pnt

def make_reference_box_at_origin(extents, color="YELLOW"):
    """
    Create a wireframe or solid reference box at origin with given extents.
    extents: [length, height, width] → corresponds to [X, Y, Z]
    """
    length, height, width = extents
    box = BRepPrimAPI_MakeBox(length, height, width).Shape()
    return box


In [18]:
# Utility to swap height and width effectively
# rotating the shape section around its X axis

def swap_width_and_height_if_required(profile_match, obb_geom):
    """
    If the classification indicates the shape is rotated (Requires_rotation),
    swap the Y and Z axes and corresponding extents, and fix handedness.

    Parameters:
        profile_match: dict with "Requires_rotation" flag
        obb_geom: dict with aligned_dir_x/y/z and aligned_extents

    Returns:
        updated obb_geom (axes and extents swapped if needed)
    """
    
    if profile_match.get("Requires_rotation", False):
        print("🔁 Profile requires axis swap (Y <-> Z) to match width/height")

        dir_x = obb_geom["aligned_dir_x"]
        dir_y = obb_geom["aligned_dir_y"]
        dir_z = obb_geom["aligned_dir_z"]
        extents = obb_geom["aligned_extents"]

        # Swap height (Y) and width (Z)
        dir_y, dir_z = dir_z, dir_y
        extents[1], extents[2] = extents[2], extents[1]

        dir_x, dir_y, dir_z = ensure_right_handed(dir_x, dir_y, dir_z)

        obb_geom["aligned_dir_x"] = dir_x
        obb_geom["aligned_dir_y"] = dir_y
        obb_geom["aligned_dir_z"] = dir_z
        obb_geom["aligned_extents"] = extents

    else:
        print("✅ No rotation required by profile")

    return obb_geom


In [19]:
# Utility function to flip channel shape if 'toes down'

def rotate_shape_if_toe_down(shape, obb_geom):
    """
    Rotates the shape 180° around the aligned X-axis (beam direction) if the
    aligned center Z-position indicates the toe is down.

    Parameters:
        shape: TopoDS_Shape — the OCC shape to rotate
        obb_geom: dict — must include:
            - 'aligned_center': gp_Pnt
            - 'aligned_extents': list of 3 floats
            - 'aligned_dir_x/y/z': gp_Dir (axes)

    Returns:
        shape (possibly rotated), updated obb_geom
    """
    from OCC.Core.gp import gp_Trsf, gp_Ax1
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

    center = obb_geom["aligned_center"]
    dir_x = obb_geom["aligned_dir_x"]
    dir_y = obb_geom["aligned_dir_y"]
    dir_z = obb_geom["aligned_dir_z"]

    half_width = obb_geom["aligned_extents"][2] / 2  # Z = flange direction
    z_local = center.Z()

    if z_local < -half_width:
        print(f"🔄 Shape is toe down (Z={z_local:.2f} < -{half_width:.2f}) — rotating 180° around X-axis")
        trsf_rotate = gp_Trsf()
        trsf_rotate.SetRotation(gp_Ax1(center, dir_x), 3.14159)

        shape = BRepBuilderAPI_Transform(shape, trsf_rotate, True).Shape()

        # Reverse Y and Z to match rotation
        dir_y.Reverse()
        dir_z.Reverse()
        dir_x, dir_y, dir_z = ensure_right_handed(dir_x, dir_y, dir_z)

        obb_geom["aligned_dir_x"] = dir_x
        obb_geom["aligned_dir_y"] = dir_y
        obb_geom["aligned_dir_z"] = dir_z

    else:
        print(f"✅ Shape is toe up (Z={z_local:.2f} ≥ -{half_width:.2f}) — no rotation needed")

    return shape, obb_geom




In [20]:
# Beam final alignment logic
def align_beam(shape, profile_match, obb_geom):
    
    obb_geom = swap_width_and_height_if_required(profile_match, obb_geom)
    return shape, obb_geom



In [21]:
# final channel alignment logic
def align_channel(shape, profile_match, obb_geom):

    obb_geom = swap_width_and_height_if_required(profile_match, obb_geom)
    shape, obb_geom = rotate_shape_if_toe_down(shape, obb_geom)

    return shape, obb_geom



In [22]:
def align_angle(shape, profile_match, obb_geom):
    print("📐 Angle alignment: TODO — implement EA/UEA logic and leg orientation")
    # For now, return as-is
    return shape, obb_geom


In [23]:
# Geometry final position despatcher
def align_shape_by_profile_type(prelimenary_aligned_shape, profile_match, obb_geom):
    shape_type = profile_match.get("Profile_type")

    if shape_type == "I":
        return align_beam(prelimenary_aligned_shape, profile_match, obb_geom)

    elif shape_type == "U":
        return align_channel(prelimenary_aligned_shape, profile_match, obb_geom)

    elif shape_type == "L":
        return align_angle(prelimenary_aligned_shape, profile_match, obb_geom)

    else:
        raise ValueError(f"❌ Unsupported shape type: '{shape_type}'")


In [24]:
if __name__ == "__main__":

    # step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    step_path = "../data/TestPFC.step"
    shape_orig = load_step(step_path)

    # visualize(shape_orig, viewer, color="GRAY")

    # STEP 1: Align to major geometry (robust PCA/face-based)
    primary_aligned_shape, trsf, cs, largest_face, dir_x, dir_y, dir_z = robust_align_solid_from_geometry(shape_orig)
    # visualize(primary_aligned_shape, viewer, color="PSS_LIGHT_BLUE")
    dir_x, dir_y, dir_z = ensure_right_handed(dir_x, dir_y, dir_z)

    # STEP 2: Compute OBB geometry
    obb_geom = compute_obb_geometry(primary_aligned_shape)
    aligned_extents = obb_geom["aligned_extents"]  # [length, height, width]
    aligned_center = obb_geom["aligned_center"]
    aligned_dir_x = obb_geom["aligned_dir_x"]
    aligned_dir_y = obb_geom["aligned_dir_y"]
    aligned_dir_z = obb_geom["aligned_dir_z"]

    # STEP 3: Compute section area
    section_area = compute_section_area(primary_aligned_shape)

    # STEP 4: Build classification signature
    cs = {
        "span_web": aligned_extents[1],
        "span_flange": aligned_extents[2],
        "area": section_area,
        "length": aligned_extents[0]
    }

    # print("Classification input:")
    # print("  span_web   =", cs["span_web"])
    # print("  span_flange=", cs["span_flange"])
    # print("  area       =", cs["area"])
    # print("  length     =", cs["length"])

    # STEP 5: Match against JSON library
    profile_match = classify_profile(cs, json_path, tol_dim=1.0, tol_area=5)
    if profile_match is None:
        raise ValueError("❌ No matching profile found")

    print(profile_match)
    
    # STEP 6: Shape-specific alignment
    aligned_shape, obb_geom = align_shape_by_profile_type(
        primary_aligned_shape,
        profile_match,
        obb_geom
    )

    # Extract post-alignment values from step 6
    aligned_extents = obb_geom["aligned_extents"]
    aligned_center = obb_geom["aligned_center"]
    aligned_dir_x = obb_geom["aligned_dir_x"]
    aligned_dir_y = obb_geom["aligned_dir_y"]
    aligned_dir_z = obb_geom["aligned_dir_z"]
    
    # # STEP 6: Handle swapped dimensions
    # swap_axes = profile_match.get("Requires_rotation", False)
    # if swap_axes:
    #     print("🔁 Swapping height/width axes to match profile classification")
    #     aligned_dir_y, aligned_dir_z = aligned_dir_z, aligned_dir_y
    #     aligned_extents[1], aligned_extents[2] = aligned_extents[2], aligned_extents[1]
        
    # ✅ Ensure handedness is maintained after swap
    aligned_dir_x, aligned_dir_y, aligned_dir_z = ensure_right_handed(aligned_dir_x, aligned_dir_y, aligned_dir_z)

    # STEP 7: Compute DSTV origin
    origin_local = compute_dstv_origin(aligned_center, aligned_extents, aligned_dir_x, aligned_dir_y, aligned_dir_z)

    # STEP 8: Align OBB to global DSTV frame using frame transform
    # Construct gp_Ax3 from origin and axis directions
    obb_frame = gp_Ax3(
        origin_local,
        aligned_dir_z,  # Y axis becomes "main" direction (Z is inferred from X × Y)
        aligned_dir_x   # X axis as the reference direction
    )
    
    aligned_shape, trsf = align_obb_to_dstv_frame(
        primary_aligned_shape,
        origin_local,
        aligned_dir_x,
        aligned_dir_y,
        aligned_dir_z
    )

    obb_final = compute_obb_geometry(aligned_shape)
    final_dir_x = obb_final["aligned_dir_x"]
    final_dir_y = obb_final["aligned_dir_y"]
    final_dir_z = obb_final["aligned_dir_z"]

    # STEP 8.5 - rcreate final location frame for use in hole positioning
    dstv_frame = gp_Ax3(
        gp_Pnt(0, 0, 0),
        gp_Dir(final_dir_z.XYZ()),  # Z as main (Y axis of DSTV)
        gp_Dir(final_dir_x.XYZ())   # X as length axis
    )
    
    # STEP 9: Visual debugging — origin and axes
    origin_marker = make_origin_marker(gp_Pnt(0, 0, 0))
    arrow_x = make_arrow(gp_Pnt(0, 0, 0), final_dir_x)  # X = length
    arrow_y = make_arrow(gp_Pnt(0, 0, 0), final_dir_y)  # Y = height
    arrow_z = make_arrow(gp_Pnt(0, 0, 0), final_dir_z)  # Z = width

    visualize(origin_marker, viewer, color="PSS_LIGHT_BLUE")
    visualize(arrow_x, viewer, color="RED")
    visualize(arrow_y, viewer, color="GREEN")
    visualize(arrow_z, viewer, color="BLUE")
    visualize(aligned_shape, viewer, color="PSS_DARK_BLUE")

    print_result_table(profile_match)


{'Designation': '100x50x10', 'Category': 'PFC', 'Profile_type': 'U', 'Match_score': 0.20230556370944988, 'Requires_rotation': True, 'JSON': {'height': 100.0, 'width': 50.0, 'csa': 1300.0, 'length': 0, 'Mass': 10.20315915751768, 'web_thickness': 5.0, 'flange_thickness': 8.5, 'root_radius': 9.0, 'toe_radius': 0.0}, 'STEP': {'height': 100.00000020000002, 'width': 50.00000020000003, 'area': 1297.3700328717778, 'length': 1000.0000001999999, 'mass': 10.20315915751768, 'web_thickness': 5.0, 'flange_thickness': 8.5, 'root_radius': 9.0, 'toe_radius': 0.0}}
🔁 Profile requires axis swap (Y <-> Z) to match width/height
⚠️ Detected left-handed system. Reversing Z to enforce right-handed convention.
✅ Shape is toe up (Z=-0.00 ≥ -25.00) — no rotation needed


,Field,STEP File,JSON Spec
0,Designation,100x50x10,
1,Category,PFC,
2,Profile Type,U,
3,Match Score,0.20,
4,Rotation Required,True,
5,Height (mm),100.00,100.00
6,Width (mm),50.00,50.00
7,CSA (mm²),1297.37,1300.00
8,Length (mm),1000.00,0.00
9,Mass (Kg),10.20,


In [25]:
# DSTV Plane projection

In [26]:
from OCC.Core.gp import gp_Pln, gp_Pnt, gp_Dir, gp_XYZ
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox

frame = dstv_frame
origin = frame.Location()
dir_x = frame.XDirection()
dir_y = frame.YDirection()
dir_z = frame.Direction()
length, height, width = obb_geom["aligned_extents"]

# Helper: build a plane face at a given origin and normal

def make_offset_face(center: gp_Pnt, normal: gp_Dir, axis_x: gp_Dir, size_x: float, size_y: float, offset: float):
    """
    Create a face centered at 'center', normal to 'normal', offset outward by 'offset',
    using 'axis_x' as the local X axis, and size_x/size_y as the face dimensions.
    """
    from OCC.Core.gp import gp_Vec, gp_Ax3, gp_Pln
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace

    # Apply offset along normal direction
    offset_vec = gp_Vec(normal) * offset
    offset_center = gp_Pnt(center.XYZ().Added(offset_vec.XYZ()))

    # Build the face on a defined plane frame
    frame = gp_Ax3(offset_center, normal, axis_x)
    plane = gp_Pln(frame)
    return BRepBuilderAPI_MakeFace(plane, -size_x/2, size_x/2, -size_y/2, size_y/2).Face()


# Configuration
visual_offset = 10.0  # mm
half_height = height / 2.0
half_width = width / 2.0
plane_center = gp_Pnt(
    origin.XYZ()
    .Added(dir_x.XYZ().Multiplied(length / 2.0))
    .Added(dir_y.XYZ().Multiplied(height / 2.0))
    .Added(dir_z.XYZ().Multiplied(width / 2.0))
)

# === FACE O: top flange, ZX plane, normal = Y+

face_o = make_offset_face(
    center=plane_center,
    normal=dir_y,  # Y+
    axis_x=dir_x,
    size_x=length,
    size_y=width,
    offset=height/2 + visual_offset
)

# === FACE U: bottom flange, ZX plane, normal = Y−

face_u = make_offset_face(
    center=plane_center,
    normal=gp_Dir(-dir_y.X(), -dir_y.Y(), -dir_y.Z()),  # Y−
    axis_x=dir_x,
    size_x=length,
    size_y=width,
    offset=height/2 + visual_offset
)

# === FACE V: web, XY plane, normal = Z+

face_v = make_offset_face(
    center=plane_center,
    normal=dir_z,
    axis_x=dir_x,
    size_x=length,
    size_y=height,
    offset=width/2 + visual_offset
)

# Visualize
visualize(face_v, viewer, color="RED", alpha=0.2)
visualize(face_o, viewer, color="GREEN", alpha=0.2)
visualize(face_u, viewer, color="BLUE", alpha=0.2)

In [27]:
# Start of hole detection

In [28]:
def format_hole_table(df):
    # Custom sort order for faces
    face_order = {"V": 0, "O": 1, "U": 2}
    df_sorted = df.copy()
    df_sorted["FaceOrder"] = df_sorted["Code"].map(face_order)
    df_sorted = df_sorted.sort_values(by=["FaceOrder", "X (mm)", "Y (mm)"]).drop(columns="FaceOrder")

    # Format columns
    fmt_cols = {
        "Diameter (mm)": "{:.1f}",
        "X (mm)": "{:.1f}",
        "Y (mm)": "{:.1f}"
    }
    valid_cols = {col: fmt for col, fmt in fmt_cols.items() if col in df_sorted.columns}

    # Style the DataFrame
    return (
        df_sorted.style
        .set_caption("Detected Holes by Face")
        .set_table_styles([{
            'selector': 'caption',
            'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]
        }])
        .format(valid_cols)
        .background_gradient(cmap="Blues", subset=["Diameter (mm)"])
    )


In [29]:
def show_projected_markers_on_dstv_planes(df_holes, origin_dstv, dstv_frame, display, radius=4.0, offset=10.0):
    from OCC.Core.gp import gp_Vec, gp_Pnt

    # DSTV axis vectors
    X = np.array([dstv_frame.XDirection().X(), dstv_frame.XDirection().Y(), dstv_frame.XDirection().Z()])
    Y = np.array([dstv_frame.YDirection().X(), dstv_frame.YDirection().Y(), dstv_frame.YDirection().Z()])
    Z = np.array([dstv_frame.Direction().X(),   dstv_frame.Direction().Y(), dstv_frame.Direction().Z()])

    origin_np = np.array([origin_dstv.X(), origin_dstv.Y(), origin_dstv.Z()])

    color_map = {
        "U": "BLUE",
        "O": "GREEN",
        "V": "RED"
    }

    for _, row in df_holes.iterrows():
        code = row["Code"]
        x = row["X (mm)"]
        y = row["Y (mm)"]

        # Local 2D to 3D (no offset yet)
        local_vec = origin_np + x * X
        
        if code == "V":
            local_vec += y * Y + (width + offset) * Z
        elif code == "O":
            local_vec += y * Z + (height + offset) * Y  # correct: O is +Y
        elif code == "U":
            local_vec += y * Z - offset * Y  # correct: U is −Y

        point = gp_Pnt(*local_vec)
        marker = make_origin_marker(point=point, radius=radius)
        visualize(marker, viewer, color=color_map.get(code, "GRAY"), alpha=1.0, clear=False)


In [30]:
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.gp import gp_Vec
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib

import numpy as np
import pandas as pd

def classify_and_project_holes_dstv(solid, dstv_frame, origin_dstv):
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    cm = props.CentreOfMass()
    centroid = np.array([cm.X(), cm.Y(), cm.Z()])

    im = props.MatrixOfInertia()
    T = np.array([[im.Value(i, j) for j in (1, 2, 3)] for i in (1, 2, 3)])
    ev, evec = np.linalg.eigh(T)
    order = np.argsort(ev)
    dirs = [gp_Vec(*evec[:, i]) for i in order]
    
    L = np.array([dstv_frame.XDirection().X(), dstv_frame.XDirection().Y(), dstv_frame.XDirection().Z()])
    F = np.array([dstv_frame.YDirection().X(), dstv_frame.YDirection().Y(), dstv_frame.YDirection().Z()])
    W = np.array([dstv_frame.Direction().X(),   dstv_frame.Direction().Y(),   dstv_frame.Direction().Z()])


    bbox = Bnd_Box()
    brepbndlib.Add(solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    spans = np.array([xmax - xmin, ymax - ymin, zmax - zmin])
    half_extents = 0.5 * np.sort(spans)[::-1]
    length, width, thickness = half_extents * 2

    origin_nc1 = np.array([origin_dstv.X(), origin_dstv.Y(), origin_dstv.Z()])


    tol_long = 0.7
    tol_web = 0.7
    hole_data = []
    explorer = TopExp_Explorer(solid, TopAbs_FACE)

    while explorer.More():
        face = topods.Face(explorer.Current())
        adaptor = BRepAdaptor_Surface(face, True)
        if adaptor.GetType() == GeomAbs_Cylinder:
            cyl = adaptor.Cylinder()
            axis_dir = np.array([cyl.Axis().Direction().X(),
                                 cyl.Axis().Direction().Y(),
                                 cyl.Axis().Direction().Z()])
            axis_dir /= np.linalg.norm(axis_dir)

            if abs(np.dot(axis_dir, L)) > tol_long:
                explorer.Next()
                continue

            pf = GProp_GProps()
            brepgprop.SurfaceProperties(face, pf)
            fc = np.array([pf.CentreOfMass().X(),
                           pf.CentreOfMass().Y(),
                           pf.CentreOfMass().Z()])

            if abs(np.dot(axis_dir, W)) > tol_web:
                code, rgb = "V", (1.0, 0.0, 0.0)
            else:
                side = np.dot(fc - centroid, F)
                code, rgb = ("O", (0.0, 1.0, 0.0)) if side > 0 else ("U", (0.0, 0.0, 1.0))

            hole_data.append((face, code, rgb, fc))
        explorer.Next()

    rows = []
    for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
        adaptor = BRepAdaptor_Surface(face, True)
        diam = 2.0 * adaptor.Cylinder().Radius()
        v = fc - origin_nc1

        if code == "V":
            x = abs(float(np.dot(v, L)))
            y = abs(float(np.dot(v, F)))
        elif code == "U":
            x = abs(float(np.dot(v, L)))
            y = abs(float(np.dot(v, W)))
        elif code == "O":
            x = abs(float(np.dot(v, L)))
            y = abs(float(np.dot(v, W)))

        rows.append({
            "Hole #":        idx,
            "Code":          code,
            "Diameter (mm)": round(diam, 2),
            "X (mm)":        round(round(x), 2),
            "Y (mm)":        round(round(y), 2)
        })

    df_holes = pd.DataFrame(rows)
    return df_holes, hole_data, origin_nc1, L, F, W


In [31]:
def check_duplicate_holes(df_holes, tolerance=0.1):
    """
    Checks for duplicate holes (same X, Y, and Code) within a given tolerance (in mm).
    Raises a warning if duplicates are found.
    """
    # Round to given tolerance (to handle small numeric noise)
    df_check = df_holes.copy()
    df_check["X_r"] = (df_check["X (mm)"] / tolerance).round().astype(int)
    df_check["Y_r"] = (df_check["Y (mm)"] / tolerance).round().astype(int)

    # Check for duplicates based on rounded X, Y, and face Code
    duplicates = df_check.duplicated(subset=["Code", "X_r", "Y_r"], keep=False)

    if duplicates.any():
        dup_df = df_holes[duplicates]
        print("⚠️ Warning: Duplicate holes detected at the same position and face:\n")
        print(dup_df[["Code", "X (mm)", "Y (mm)", "Diameter (mm)"]])
    else:
        print("✅ No duplicate holes found.")


In [32]:

raw_df_holes, hole_data, origin_nc1, L, F, W = classify_and_project_holes_dstv(aligned_shape, dstv_frame, origin)
check_duplicate_holes(raw_df_holes, tolerance=0.5)  # 0.5mm tolerance
show_projected_markers_on_dstv_planes(raw_df_holes, origin, dstv_frame, viewer)
df_holes = format_hole_table(raw_df_holes)
display(df_holes)


✅ No duplicate holes found.


,Hole #,Code,Diameter (mm),X (mm),Y (mm)
3,4,V,10.0,100.0,50.0
2,3,O,10.0,50.0,25.0
1,2,U,10.0,50.0,25.0
0,1,U,10.0,100.0,25.0


In [33]:

step_vals = profile_match["STEP"]

header_dict = {
    "Designation":       profile_match["Designation"],
    "Mass":              step_vals["mass"],
    "Height":            step_vals["height"],
    "Width":             step_vals["width"],
    "CSA":               step_vals["area"],
    "web_thickness":     step_vals["web_thickness"],
    "flange_thickness":  step_vals["flange_thickness"],
    "root_radius":       step_vals["root_radius"],
    "toe_radius":        step_vals["toe_radius"],
    "code_profile":      profile_match["Profile_type"],
    "Length":            length
}

# %%
"""
Jupyter Notebook: Write DSTV NC1 ST-block header to .nc1 file with specified order
"""
# Cell 1: Define DSTV header fields inline
# Replace placeholder values with your actual data
project_number     = 'PROJ-123'
out_filename        = '0444-1 20250606'
model_filename     = 'model.step'
material_grade     = 'S355JR'
quantity           = 1

faces = ['V', 'U', 'O']  #Face priority order

# %%

filename = f"{nc_path}{out_filename}.nc1"
with open(filename, 'w') as f:
    f.write('ST\n')
    f.write(f"  Project Number\n")
    f.write(f"  Drawing Number\n")
    f.write(f"  {out_filename}\n")
    f.write(f"  {model_filename}\n")
    f.write(f"  {material_grade}\n")
    f.write(f"  {quantity}\n")
    f.write(f"  {header_dict['Designation']}\n")
    f.write(f"  {header_dict['code_profile']}\n")
    f.write(f"    {header_dict['Length']:8.2f}\n")
    f.write(f"    {header_dict['Height']:8.2f}\n")
    f.write(f"    {header_dict['Width']:8.2f}\n")
    f.write(f"    {header_dict['flange_thickness']:8.2f}\n")
    f.write(f"    {header_dict['web_thickness']:8.2f}\n")
    f.write(f"    {header_dict['root_radius']:8.2f}\n")
    f.write(f"    {header_dict['Mass']:8.2f}\n")
    f.write('        0.00\n') #surface area
    # Following the spec, three zeros
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    
    # BO blocks by face
    for face in faces:
        df_face = raw_df_holes[raw_df_holes['Code'] == face]
        if df_face.empty:
            continue
        f.write('BO\n')
        for _, row in df_face.iterrows():
            x = row['X (mm)']
            y = row['Y (mm)']
            d = row['Diameter (mm)']
            # Right-align numeric columns: x, y (8-wide), diameter (6-wide)
            f.write(f"  {face.lower()}  {x:8.2f} {y:8.2f} {d:6.2f}\n")
            
    f.write('EN\n')
print(f"DSTV header written to {filename}")

DSTV header written to ../data/output/nc1/0444-1 20250606.nc1


In [34]:
# Drawing

In [35]:

# Final integrated function with border, embedded logo, title block, and enhanced layout

def generate_hole_projection_html(
    df_holes,
    header_dict,
    project_number,
    out_filename,
    model_filename,
    material_grade,
    quantity,
    logo_path,
    drawing_file="Drawing.html"
):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from io import BytesIO
    from base64 import b64encode
    import pandas as pd
    
    # Prepare image and drawing
    df = df_holes.copy()
    df_sorted = df.sort_values(by=["Code", "Diameter (mm)", "X (mm)", "Y (mm)"]).copy()
    df_sorted["Hole ID"] = (
        df_sorted.groupby("Code").cumcount() + 1
    ).astype(str).str.zfill(2)
    df_sorted["ID"] = df_sorted["Code"] + "-" + df_sorted["Hole ID"].astype(str)

    x_min, x_max = df_sorted["X (mm)"].min(), df_sorted["X (mm)"].max()
    y_min, y_max = df_sorted["Y (mm)"].min(), df_sorted["Y (mm)"].max()

    def plot_face(face_code, ax, title):
        face_df = df_sorted[df_sorted["Code"] == face_code]
        ax.set_title(f"{title} View ({face_code} Face)", fontsize=10)
        ax.set_aspect('equal')
        x_min, x_max = df_sorted["X (mm)"].min(), df_sorted["X (mm)"].max()
        y_min, y_max = df_sorted["Y (mm)"].min(), df_sorted["Y (mm)"].max()
        ax.set_xlim(x_min - 100, x_max + 100)
        ax.set_ylim(y_max + 50, y_min - 50)
        ax.set_xlabel("X (mm)")
        ax.set_ylabel("Y (mm)")
        ax.grid(True, linestyle="--", linewidth=0.5)
    
        for i, (_, row) in enumerate(face_df.iterrows()):
            x, y = row["X (mm)"], row["Y (mm)"]
            ax.plot(x, y, marker='+', color='black', markersize=8, mew=1.5)
            dy = -30 if i % 2 == 0 else 20  # alternate label position
            ax.text(x, y + dy, row["ID"], fontsize=6, ha='center')

    # Create drawing
    fig, axs = plt.subplots(3, 1, figsize=(12, 9))
    for ax, face, title in zip(axs, ["O", "V", "U"], ["Top", "Middle", "Bottom"]):
        plot_face(face, ax, title)
    plt.tight_layout()

    # Convert figure to base64
    buf = BytesIO()
    plt.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    buf.seek(0)
    img_base64 = b64encode(buf.read()).decode("utf-8")
    plt.close(fig)

    # Convert logo to base64
    with open(logo_path, "rb") as f:
        logo_base64 = b64encode(f.read()).decode("utf-8")
    logo_data_url = f"data:image/png;base64,{logo_base64}"

    # Title block HTML
    title_block_html = f"""
    <table style="width:100%; border: 1px solid black; border-collapse: collapse; font-size: 14px; margin: 0;">
      <tr>
        <td rowspan="4" style="border: 1px solid black; text-align:center; width: 160px;">
          <img src="{logo_data_url}" style="max-height:100px; max-width:140px;" alt="Logo" />
        </td>
        <td style="border: 1px solid black;"><strong>Project</strong></td>
        <td style="border: 1px solid black;">{project_number}</td>
        <td style="border: 1px solid black;"><strong>NC File</strong></td>
        <td style="border: 1px solid black;">{out_filename}</td>
      </tr>
      <tr>
        <td style="border: 1px solid black;"><strong>Designation</strong></td>
        <td style="border: 1px solid black;">{header_dict['Designation']}</td>
        <td style="border: 1px solid black;"><strong>Length</strong></td>
        <td style="border: 1px solid black;">{header_dict['Length']:8.0f}mm</td>
      </tr>
      <tr>
        <td style="border: 1px solid black;"><strong>Mass</strong></td>
        <td style="border: 1px solid black;">{header_dict['Mass']:8.2f}kg</td>
        <td style="border: 1px solid black;"><strong>Material</strong></td>
        <td style="border: 1px solid black;">{material_grade}</td>
      </tr>
      <tr>
        <td style="border: 1px solid black;"><strong>Quantity</strong></td>
        <td style="border: 1px solid black;">{quantity}</td>
        <td style="border: 1px solid black;"><strong>Doc Ref</strong></td>
        <td style="border: 1px solid black;">{drawing_file}</td>
      </tr>
    </table>
    """


    html = f"""
    <html>
    <head>
    <style>
      body {{
        font-family: monospace;
        margin: 0;
        padding: 0;
        border: 4px solid black;
      }}
    
      img {{
        display: block;
        margin: 0 auto;
        max-width: 100%;
        width: 100%;
      }}
    
      h2 {{
        text-align: center;
      }}
    
      table {{
        width: 100%;
        border-collapse: collapse;
        font-size: 14px;
      }}
    
      th, td {{
        border: 1px solid black;
        padding: 4px 8px;
        text-align: center;
      }}
    
      @media print {{
        img {{
          page-break-after: always;
        }}
      }}
    </style>
    </head>
    <body>
    <style>
        body {{
            font-family: monospace;
            border: 4px solid black;
            padding: 20px;
            margin: 20px;
        }}
        img {{
            width: 100%;
            max-width: 1000px;
            display: block;
            margin: auto;
        }}
        h2 {{
            text-align: center;
        }}
        table {{
            width: 100%;
            margin: 20px auto;
            border-collapse: collapse;
        }}
        th, td {{
            border: 1px solid black;
            padding: 4px 8px;
            text-align: center;
        }}
        @media print {{
            img {{
                page-break-after: always;
            }}
        }}
    </style>
    </head>
    <body>
        <h2>{drawing_file} Drilling Schedule</h2>
        <img src="data:image/png;base64,{img_base64}" />
    """

    for face in ["O", "V", "U"]:
        face_df = df_sorted[df_sorted["Code"] == face][["ID", "X (mm)", "Y (mm)", "Diameter (mm)"]].reset_index(drop=True)
        html += f"<h2>Hole Table - {face} Face</h2><table><tr><th>ID</th><th>X (mm)</th><th>Y (mm)</th><th>Diameter (mm)</th></tr>"
        for _, row in face_df.iterrows():
            html += f"<tr><td>{row['ID']}</td><td>{row['X (mm)']}</td><td>{row['Y (mm)']}</td><td>{row['Diameter (mm)']}</td></tr>"
        html += "</table>"

    html += title_block_html
    html += "</body></html>"

    # Ensure output directory exists
    os.makedirs(os.path.dirname(drawing_path), exist_ok=True)
    
    # Write to file
    with open(f"{drawing_path}{drawing_file}", "w") as f:
        f.write(html)
    
    print(f"✅ Full hole drawing saved to: {drawing_file}")



In [36]:
# generate_hole_projection_html(
#     df_holes,
#     header_dict,
#     project_number,
#     out_filename,
#     model_filename,
#     material_grade,
#     quantity,
#     logo_path,
#     output_path="Drawing.html"
# )

generate_hole_projection_html(
    raw_df_holes,
    header_dict,
    "19999",
    f"19999-0444-1",
    "0444-1.step",
    "S355",
    1,
    f"{media_path}PSS_Standard_RGB.png",
    f"19999-MEC-0001.html"
)

✅ Full hole drawing saved to: 19999-MEC-0001.html


In [37]:
# end face analysis

In [38]:
import math
import numpy as np
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Ax3

def get_face_center(face):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    pnt = props.CentreOfMass()
    return np.array([pnt.X(), pnt.Y(), pnt.Z()])

def get_face_normal(face, u=0.5, v=0.5):
    surface = BRepAdaptor_Surface(face, True)
    gp_pnt = gp_Pnt()
    gp_vec_u = gp_Vec()
    gp_vec_v = gp_Vec()
    surface.D1(u, v, gp_pnt, gp_vec_u, gp_vec_v)
    normal_vec = gp_vec_u.Crossed(gp_vec_v)
    return np.array([normal_vec.X(), normal_vec.Y(), normal_vec.Z()])

def angle_between(v1, v2):
    cos_theta = np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1.0, 1.0)
    return math.degrees(math.acos(cos_theta))

def analyze_end_faces_from_frame(solid: TopoDS_Shape, ax3: gp_Ax3, tol=1e-3):
    axis_x = np.array([ax3.Direction().X(), ax3.Direction().Y(), ax3.Direction().Z()])
    axis_y = np.array([ax3.YDirection().X(), ax3.YDirection().Y(), ax3.YDirection().Z()])
    axis_z = np.array([ax3.XDirection().X(), ax3.XDirection().Y(), ax3.XDirection().Z()])

    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    face_data = []

    while explorer.More():
        face = explorer.Current()
        center = get_face_center(face)
        normal = get_face_normal(face)
        dot = np.dot(center, axis_x)
        face_data.append((face, center, normal, dot))
        explorer.Next()

    if not face_data:
        return []

    start_face_data = min(face_data, key=lambda fd: fd[3])
    end_face_data = max(face_data, key=lambda fd: fd[3])

    results = []
    for label, (face, center, normal, dot) in zip(["start", "end"], [start_face_data, end_face_data]):
        n_unit = normal / np.linalg.norm(normal)
        angle_to_x = angle_between(n_unit, axis_x)

        if abs(angle_to_x) < 1.0:
            results.append({
                "end": label,
                "is_perpendicular": True,
                "slope_type": "none",
                "angle_y_deg": 0.0,
                "angle_z_deg": 0.0
            })
        else:
            proj_y = np.dot(n_unit, axis_y)
            proj_z = np.dot(n_unit, axis_z)
            angle_y = math.degrees(math.atan2(proj_y, np.dot(n_unit, axis_x)))
            angle_z = math.degrees(math.atan2(proj_z, np.dot(n_unit, axis_x)))
            
            slope_type = "compound" if abs(proj_y) > tol and abs(proj_z) > tol else \
                         "slope_y" if abs(proj_y) > tol else \
                         "slope_z" if abs(proj_z) > tol else "unknown"

            results.append({
                "end": label,
                "is_perpendicular": False,
                "slope_type": slope_type,
                "angle_y_deg": round(angle_y, 2),
                "angle_z_deg": round(angle_z, 2)
            })

    return results


In [39]:

end_results = analyze_end_faces_from_frame(aligned_shape, dstv_frame)

for r in end_results:
    print(r)


{'end': 'start', 'is_perpendicular': False, 'slope_type': 'unknown', 'angle_y_deg': 180.0, 'angle_z_deg': 180.0}
{'end': 'end', 'is_perpendicular': True, 'slope_type': 'none', 'angle_y_deg': 0.0, 'angle_z_deg': 0.0}
